https://github.com/viveknaidu007/repo-240-CI-CD-with-github-actions.git

# deploying a React (Vite) frontend and a Flask backend, you should set up CI/CD using GitHub Actions to automatically deploy updates. Here’s how you can structure your GitHub Actions 

Since you are deploying a **React (Vite) frontend** and a **Flask backend**, you should set up **CI/CD using GitHub Actions** to automatically deploy updates. Here’s how you can structure your GitHub Actions workflow for **continuous deployment**.

---

## **Deployment Plan**
### 🔹 **Backend (Flask)**
- Hosted on **AWS EC2** or **Docker (AWS ECS)**
- Uses **Gunicorn + Nginx**
- CI/CD triggers on backend code changes
- Restarts the server after deployment

### 🔹 **Frontend (React + Vite)**
- Hosted on **Vercel, Netlify, or AWS S3 + CloudFront**
- CI/CD triggers on frontend code changes
- Deploys the updated frontend automatically

---

## **Step 1: Configure GitHub Actions**
In your GitHub repository, create a **`.github/workflows/deploy.yml`** file.

```yaml
name: Deploy Backend and Frontend

on:
  push:
    branches:
      - main  # Change to your deployment branch

jobs:
  backend-deploy:
    name: Deploy Flask Backend
    runs-on: ubuntu-latest

    steps:
      - name: Checkout Repository
        uses: actions/checkout@v3

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.10'

      - name: Install Dependencies
        run: |
          cd backend
          pip install -r requirements.txt

      - name: Deploy to EC2
        env:
          PRIVATE_KEY: ${{ secrets.SSH_PRIVATE_KEY }}
          HOST: ${{ secrets.EC2_HOST }}
          USER: ${{ secrets.EC2_USER }}
        run: |
          echo "$PRIVATE_KEY" > private_key.pem
          chmod 600 private_key.pem
          scp -i private_key.pem -r backend/* $USER@$HOST:/home/$USER/backend/
          ssh -i private_key.pem $USER@$HOST 'cd /home/$USER/backend && sudo systemctl restart gunicorn'

  frontend-deploy:
    name: Deploy React Frontend
    runs-on: ubuntu-latest

    steps:
      - name: Checkout Repository
        uses: actions/checkout@v3

      - name: Set up Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '18'

      - name: Install Dependencies
        run: |
          cd frontend
          npm install

      - name: Build Frontend
        run: |
          cd frontend
          npm run build

      - name: Deploy to Vercel
        run: |
          npm i -g vercel
          vercel --token ${{ secrets.VERCEL_TOKEN }} --prod
```

---

## **Step 2: Setup Secrets in GitHub**
Go to **GitHub → Settings → Secrets → Actions** and add these secrets:
- `SSH_PRIVATE_KEY`: Your **EC2 private key** for SSH
- `EC2_HOST`: Your **EC2 instance public IP**
- `EC2_USER`: Your **EC2 username (e.g., ubuntu)**
- `VERCEL_TOKEN`: **Vercel API token** (for frontend deployment)

---

## **Step 3: Configure EC2 for Auto-Restart**
On your EC2 instance:
1. Install **Gunicorn**:
   ```sh
   pip install gunicorn
   ```
2. Create a **Gunicorn service** (`/etc/systemd/system/gunicorn.service`):
   ```ini
   [Unit]
   Description=Gunicorn server
   After=network.target

   [Service]
   User=ubuntu
   Group=ubuntu
   WorkingDirectory=/home/ubuntu/backend
   ExecStart=/usr/bin/gunicorn --workers 4 --bind 0.0.0.0:8000 app:app

   [Install]
   WantedBy=multi-user.target
   ```
3. Start and enable the service:
   ```sh
   sudo systemctl start gunicorn
   sudo systemctl enable gunicorn
   ```

---

## **Step 4: Configure Nginx for Flask**
On **EC2**, set up Nginx as a reverse proxy:
1. Install Nginx:
   ```sh
   sudo apt update && sudo apt install nginx -y
   ```
2. Configure Nginx (`/etc/nginx/sites-available/backend`):
   ```nginx
   server {
       listen 80;
       server_name your_domain_or_ip;

       location / {
           proxy_pass http://127.0.0.1:8000;
           proxy_set_header Host $host;
           proxy_set_header X-Real-IP $remote_addr;
           proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
       }
   }
   ```
3. Enable the configuration:
   ```sh
   sudo ln -s /etc/nginx/sites-available/backend /etc/nginx/sites-enabled
   sudo systemctl restart nginx
   ```

---

## **Step 5: Deploy Frontend on Vercel**
1. Install **Vercel CLI**:
   ```sh
   npm i -g vercel
   ```
2. Link the project:
   ```sh
   cd frontend
   vercel
   ```
3. Set environment variables on **Vercel**.

---

## **How CI/CD Works**
- When you **push changes** to GitHub:
  ✅ **Backend updates** → Deploys to EC2  
  ✅ **Frontend updates** → Deploys to Vercel  
- **Zero downtime** (server auto-restarts)  
- **Production-ready deployment**

---

🚀 **Your Flask + React app now has automated deployments!**  

# ************************************************************